### Вспоминаем генераторы

In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    for i in range(2, max_number):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
        yield fib_2

fibs = generate_fib(10)

print(next(fibs))
print(next(fibs))
print(next(fibs))
print(next(fibs))


1
1
2
3


In [ ]:
fibs = generate_fib(10)
for x in fibs:
    print(x)
print('-'*10)
for x in fibs:
    print(x)

1
1
2
3
5
8
13
21
34
55
----------


Кстати, а как вернуть значение из генератора?

In [ ]:
next(fibs)

StopIteration: 

In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    return 4

fibs = generate_fib(10)
for x in fibs:
    print(x)

1
1


In [ ]:
next(fibs)

StopIteration: 

In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    return 4

fibs = generate_fib(10)

try:
    for x in fibs:
        print(x)
except StopIteration as e:
    print("Return value:", e.value)


1
1


In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    return 4

fibs = generate_fib(10)

while True:
    try:
        value = next(fibs)
        print(value)
    except StopIteration as e:
        print("Return value:", e.value)  # Выводим значение return
        break


1
1
Return value: 4


### Генераторы как корутины

Генераторы можно использовать как корутины, которые могут не только возвращать значения, но и принимать их. Это стало основой для использования генераторов в асинхронных сценариях.



In [ ]:
def coroutine():
    print("Start")
    value = yield
    print(f"Received: {value}")
    yield "Done"

coro = coroutine()

next(coro)

coro.send("Hello")


Start
Received: Hello


'Done'

Чуть более сложный пример с циклом внутри

In [ ]:
def running_total():
    total = 0
    while True:
        number = yield total
        if number is None:
            break
        total += number
    return total

gen = running_total()

next(gen)

print(gen.send(5))
print(gen.send(10))
print(gen.send(3))

gen.send(None)


105
115
118


StopIteration: 

В этом примере генератор аккумулирует суммы, принимая данные через `send()`. Такой подход можно использовать для отслеживания состояния в течение выполнения, что напоминает принципы асинхронного программирования, где состояния задач могут изменяться во времени.



Более сложный пример с обработкой исключений в генераторах:

In [ ]:
def exception_handling_coroutine():
    print("Starting coroutine")
    try:
        while True:
            try:
                value = yield
            except ValueError:
                print("ValueError caught inside coroutine!")
            else:
                print(f"Received value: {value}")
    finally:
        print("Coroutine terminating")

coro = exception_handling_coroutine()
next(coro)

coro.send(10)
coro.send(20)

coro.throw(ValueError)

coro.close()


Starting coroutine
Received value: 10
Received value: 20
ValueError caught inside coroutine!
Coroutine terminating


### yield from

Когда `yield from` применяется к подгенератору, он последовательно возвращает все значения этого подгенератора в внешний генератор.


In [ ]:
def accumulator():
    total = 0
    for i in range(3):
        total += i
        yield i
    return total

def main_generator():
    result = yield from accumulator()
    print("Accumulated total:", result)

for value in main_generator():
    print("Yielded:", value)


Yielded: 0
Yielded: 1
Yielded: 2
Accumulated total: 3


Более сложный пример с несколькими подгенераторами

In [ ]:
def numbers():
    yield 1
    yield 2
    yield 3
    return "Numbers done"

def letters():
    yield 'A'
    yield 'B'
    yield 'C'
    return "Letters done"

def main_generator():
    result1 = yield from numbers()
    print("First subgenerator result:", result1)

    result2 = yield from letters()
    print("Second subgenerator result:", result2)

    return "All subgenerators done"

for value in main_generator():
    print("Yielded:", value)


Yielded: 1
Yielded: 2
Yielded: 3
First subgenerator result: Numbers done
Yielded: A
Yielded: B
Yielded: C
Second subgenerator result: Letters done


А теперь давайте перемешаем порядок обращения к подгенераторам!

In [ ]:
def numbers():
    yield 1
    yield 2
    yield 3
    yield 4
    return "Numbers done"

def letters():
    yield 'A'
    yield 'B'
    yield 'C'
    return "Letters done"


def interleaved_generator():
    gens = [numbers(), letters()]
    results = []

    while gens:
        for gen in gens.copy():
            try:
                value = next(gen)
                yield value
            except StopIteration as e:
                results.append(e.value)
                gens.remove(gen)

    for i, result in enumerate(results, start=1):
        print(f"Subgenerator {i} result:", result)

for value in interleaved_generator():
    print("Yielded:", value)


Yielded: 1
Yielded: A
Yielded: 2
Yielded: B
Yielded: 3
Yielded: C
Yielded: 4
Subgenerator 1 result: Letters done
Subgenerator 2 result: Numbers done


In [ ]:
from collections import deque

def numbers():
    yield 1
    yield 2
    yield 3
    yield 4
    return "Numbers done"

def letters():
    yield 'A'
    yield 'B'
    yield 'C'
    return "Letters done"

def interleaved_generator():
    gens = deque([numbers(), letters()])
    results = []

    while gens:
        gen = gens.popleft()
        try:
            value = next(gen)
            yield value
            gens.append(gen)
        except StopIteration as e:
            results.append(e.value)

    for i, result in enumerate(results, start=1):
        print(f"Subgenerator {i} result:", result)

for value in interleaved_generator():
    print("Yielded:", value)


Yielded: 1
Yielded: A
Yielded: 2
Yielded: B
Yielded: 3
Yielded: C
Yielded: 4
Subgenerator 1 result: Letters done
Subgenerator 2 result: Numbers done


## Теория асинхронности и ее задачи

### Введение в асинхронность

Асинхронное программирование — это парадигма разработки, позволяющая выполнять несколько задач одновременно, не блокируя основное приложение. Это особенно полезно в задачах, где программа часто ожидает завершения операций ввода-вывода, таких как:
- Загрузка данных из интернета,
- Чтение и запись файлов,
- Взаимодействие с базами данных.

Асинхронное программирование помогает организовать выполнение так, чтобы задачи могли переключаться между состояниями ожидания и активного выполнения, что позволяет повысить производительность и использовать доступные ресурсы.

#### Асинхронность vs многопоточность vs многопроцессность

Асинхронность отличается от других подходов к параллельности:

1. **Асинхронность**:
   - Использует одну операционную систему или один поток, переключая задачи между состоянием ожидания и выполнения.
   - Применяется для I/O-bound задач, где большая часть времени уходит на ожидание данных от внешних систем.

2. **Многопоточность**:
   - Позволяет программе запускать несколько потоков одновременно. Каждый поток — это отдельный путь выполнения внутри одного процесса.
   - Подходит для задач с параллельным выполнением, но в Python многопоточность ограничена **Global Interpreter Lock (GIL)**, который не позволяет исполнять Python-код параллельно на нескольких ядрах процессора.
   
3. **Многопроцессность**:
   - Включает запуск нескольких процессов, каждый из которых имеет собственный интерпретатор Python, что позволяет исполнять код параллельно.
   - Многопроцессность обходит ограничения GIL и используется для CPU-bound задач (интенсивных по вычислениям).



#### Примеры из жизни

Чтобы лучше понять, чем асинхронность отличается от других способов многозадачности, представьте несколько сценариев.

1. **Асинхронность как планировщик задач на вечеринке**:
   - Представьте вечеринку, где один человек (ассистент) обслуживает всех гостей. Каждый гость просит его что-то выполнить (налить воды, подать еду, показать путь до туалета). Вместо того чтобы оставаться с одним гостем до конца выполнения его просьбы, ассистент обходит каждого, запоминая просьбы и возвращаясь, когда у него есть время. В итоге все получают помощь, но ассистент не тратит много времени на одного человека, оставляя других без внимания.
   - Этот подход похож на асинхронность, где цикл событий переключается между задачами, обрабатывая их по мере готовности, и не простаивает в ожидании.

2. **Многопоточность как множество официантов в ресторане**:
   - Представьте ресторан, где каждый официант обслуживает отдельный стол. У каждого официанта есть свой собственный столик, за который он отвечает, и он выполняет всю работу для него, включая прием заказа и подачу блюд.
   - Это похоже на многопоточность, где у каждой задачи (столика) есть отдельный поток, и он полностью занят ее обслуживанием. Однако в Python есть **GIL** (глобальная блокировка интерпретатора), которая накладывает ограничения на многопоточность.

3. **Многопроцессность как отдельные кухни для каждого заказа**:
   - Представьте, что ресторан разделил каждую кухню на отдельные зоны, где готовится только один заказ. Это требует больших ресурсов (поваров, пространства), но каждый заказ выполняется без помех и параллельно.
   - Многопроцессность работает аналогично, так как каждый процесс выполняется независимо от других. Python создает для каждого процесса отдельный интерпретатор и память, что позволяет обойти GIL и выполнять вычислительные задачи параллельно, но при этом увеличивает потребление ресурсов.


### Асинхронность, многопоточность и многопроцессность: детальное сравнение


| Характеристика            | Асинхронность                            | Многопоточность                        | Многопроцессность                        |
|---------------------------|------------------------------------------|----------------------------------------|------------------------------------------|
| **Подходит для**          | Задач с длительным I/O                   | Смешанных задач (I/O и CPU), но с ограничениями из-за GIL | Задач с высокой нагрузкой на CPU (CPU-bound) |
| **Особенности реализации**| Работает в одном потоке и переключается между задачами | Создает потоки, каждый из которых работает с GIL | Создает отдельные процессы с независимыми интерпретаторами |
| **Потребление ресурсов**  | Низкое потребление ресурсов              | Среднее потребление                    | Высокое потребление (каждый процесс требует отдельной памяти) |
| **Использование GIL**     | Не зависит от GIL                        | Ограничен GIL                          | GIL отсутствует, так как процессы независимы |
| **Типичная библиотека**   | `asyncio`, `aiohttp`                     | `threading`, `concurrent.futures`      | `multiprocessing`, `concurrent.futures`  |
